In [2]:
from gamspy import (
    Container,
    Set,
    Alias,
    Parameter,
    Variable,
    Equation,
    Sum,
    Model,
    Sense,
)
import pandas as pd
from itertools import product
import plotly.express as px

In [3]:
products = ["Product_A", "Product_B", "Product_C"]
time_periods = [1, 2, 3, 4]
resources = ["Resource_A", "Resource_B"]

kj = pd.DataFrame(product(products, resources))

demand_data = pd.DataFrame(
    {
        "Product_A": {1: 100, 2: 150, 3: 120, 4: 180},
        "Product_B": {1: 80, 2: 100, 3: 90, 4: 120},
        "Product_C": {1: 50, 2: 60, 3: 70, 4: 80},
    }
).unstack()

setup_cost_data = pd.DataFrame(
    [("Product_A", 100), ("Product_B", 200), ("Product_C", 300)]
)
holding_cost_data = pd.DataFrame(
    [("Product_A", 0.2), ("Product_B", 0.1), ("Product_C", 0.6)]
)

capacity_data = pd.DataFrame(
    [
        ("Resource_A", 1, 340),
        ("Resource_B", 1, 340),
        ("Resource_A", 2, 330),
        ("Resource_B", 2, 330),
        ("Resource_A", 3, 300),
        ("Resource_B", 3, 300),
        ("Resource_A", 4, 380),
        ("Resource_B", 4, 380),
    ]
)

In [7]:
holding_cost_data

,0,1
0,Product_A,0.2
1,Product_B,0.1
2,Product_C,0.6


In [9]:
# Container definition
m = Container()

# SETS
k = Set(m, name="k", description="products", records=products)
j = Set(m, name="j", description="resources", records=resources)
t = Set(m, name="t", description="time periods", records=time_periods)
KJ = Set(
    m,
    name="KJ",
    domain=[k, j],
    description="products k that can be handled by resource j",
    records=kj,
)

# ALIAS
tau = Alias(m, name="tau", alias_with=t)

# PARAMETERS
d = Parameter(
    m,
    name="d",
    domain=[k, t],
    description="demand of product k in period t",
    records=demand_data,
)
s = Parameter(
    m,
    name="s",
    domain=k,
    description="fixed setup cost for product k",
    records=setup_cost_data,
)
h = Parameter(
    m,
    name="h",
    domain=k,
    description="holding cost for product k",
    records=holding_cost_data,
)
c = Parameter(
    m,
    name="c",
    domain=[j, t],
    description="production capacity of resource j in period t",
    records=capacity_data,
)

# VARIABLES
X = Variable(
    m,
    name="X",
    domain=[k, t],
    type="positive",
    description="lot size of product k in period t",
)
Y = Variable(
    m,
    name="Y",
    domain=[k, t],
    type="binary",
    description="indicates if product k is manufactured in period t",
)
Z = Variable(
    m,
    name="Z",
    domain=[k, t],
    type="positive",
    description="stock of product k in period t",
)

In [10]:
d

Parameter(name=d, domain=[Set(name=k, domain=['*']), Set(name=t, domain=['*'])])